# Main file for thesis project

Ie the top level script which runs the whole `pic -> recognise -> aim gimbal -> repeat` show

In [ ]:
#!/usr/bin/python3
# top level script to run the show
import time
import PIL.Image

import image_classifier
import gimbal_control as gc

## A Basic Loop

In [ ]:
IC = image_classifier.ImageClassifier(
    graph_file='../Models/MobileNet_SSD_caffe/graph',
    label_file='../Models/MobileNet_SSD_caffe/categories.txt',
    class_of_interest='person')

In [ ]:
while True:
    img_array = IC.take_picture_and_start_inference()
    angles = gc.get_motor_angles()  # get the current motor angles - need these to calculate offsets
    output_dict, bb, bb_angles, inference_time_ms = IC.get_inference_result()

    if bb_angles == -1:
        print('-', end='')
        img = PIL.Image.fromarray(img_array)
        
    else:
        print('+', end='')
        # draw a box on the object
        (x1, y1), (x2, y2) = bb
        img = PIL.Image.fromarray(img_array)
        img = PIL.Image.fromarray(image_classifier.draw_bounding_box(y1, x1, y2, x2, img_array))
#         plt.imshow(img)
#         plt.show()

        # convert angles of object to centroids
        (phi_x1, phi_y1), (phi_x2, phi_y2) = bb_angles
        phi_yaw = (phi_x1 + phi_x2)/2
        phi_pitch = (phi_y1 + phi_y2)/2
        
        angles['pitch'] = -angles['pitch']
#         print('gimbal = %d, obj = %d' % (angles['pitch'], phi_pitch))
            
#         print('phi_yaw = %d' % phi_yaw)
        gc.send_angle_command(0,
                              0,
                              phi_yaw + angles['yaw'])
        # old integrator approach
#         x_target += (x1+x2)/2
#         x_target = max(-20, min(20, x_target))
#         gc.send_angle_command(0, 0, x_target)
#     time.sleep(0.05)

In [ ]:
img_array = IC.take_picture_and_start_inference()
output_dict, bb, bb_angles, inference_time_ms = IC.get_inference_result()

if bb_angles == -1:
    print('-', end='')
    img = PIL.Image.fromarray(img_array)

else:
    print('+', end='')
    # draw a box on the object
    (x1, y1), (x2, y2) = bb
    img = PIL.Image.fromarray(img_array)
    img = PIL.Image.fromarray(image_classifier.draw_bounding_box(y1, x1, y2, x2, img_array))

img